In [ ]:
import time as time
import pandas as pd
import os

from selenium import webdriver 
from selenium.webdriver.support.ui import Select

In [ ]:
url = 'http://www.cpsnl.ca/WEB/CPSNL/PhysicianSearch/Physician_Search.aspx'
df = pd.DataFrame(columns={'Last Name', 'First Name', 'Middle Name', 'Address', 'Member Type', 'Register', 'Specialty'})

In [ ]:
option = webdriver.ChromeOptions()
option.add_argument('-incognito')

browser = webdriver.Chrome(executable_path='../chromedriver', chrome_options=option)

In [ ]:
browser.get(url)

In [ ]:
# Grab the Area of Practice drop down menu 
select = Select(browser.find_element_by_id('ctl01_TemplateBody_WebPartManager1_gwpciNewQueryMenuCommon_ciNewQueryMenuCommon_ResultsGrid_Sheet0_Input4_DropDown1'))

# Loop through each Area of Practices grabbing info
for i in range(1,len(select.options)):
    
    # Regrab the Area of Practice when the page loads 
    select = Select(browser.find_element_by_id('ctl01_TemplateBody_WebPartManager1_gwpciNewQueryMenuCommon_ciNewQueryMenuCommon_ResultsGrid_Sheet0_Input4_DropDown1'))
    
    # Grab the current area of practice string to be used for Dr specialty
    specialty = select.options[i].text
    
    # Select the current area of practice
    select.select_by_index(i)
    
    # Click Find button
    browser.find_element_by_id('ctl01_TemplateBody_WebPartManager1_gwpciNewQueryMenuCommon_ciNewQueryMenuCommon_ResultsGrid_Sheet0_SubmitButton').click()
    time.sleep(3)
    
    # Try and click on the show all button if not already clicked
    try:
        # Click on show all button
        browser.find_element_by_xpath("//*[contains(text(), 'show all')]").click()
        time.sleep(2)
    except:
        print('')
    
    # Grab the Table with all the doctors
    table = browser.find_element_by_xpath("//table[@id='ctl01_TemplateBody_WebPartManager1_gwpciNewQueryMenuCommon_ciNewQueryMenuCommon_ResultsGrid_Grid1_ctl00']")
    
    # Grab all the table rows in that table
    trList = table.find_elements_by_tag_name('tr')
    
    # Remove the header
    trList = trList[1:]
    
    # Grab the table info if there are doctors in that specialty
    try:
        # Grab info for each row
        for tr in trList:
            lastName = tr.find_elements_by_tag_name('td')[0].text
            firstName = tr.find_elements_by_tag_name('td')[1].text
            middleName = tr.find_elements_by_tag_name('td')[2].text
            address = tr.find_elements_by_tag_name('td')[3].text
            memberType = tr.find_elements_by_tag_name('td')[4].text
            register = tr.find_elements_by_tag_name('td')[5].text
            
            # package the data into a dictionary
            data = {'Last Name' : lastName, 
              'First Name' : firstName, 
              'Middle Name' : middleName, 
              'Address' : address, 
              'Member Type' : memberType, 
              'Register' : register,
              'Specialty' : specialty}
            
            # Append it into the dataframe
            df = df.append(data, ignore_index=True)
    # If there are no doctors in that specialty just go to the next specialty then 
    except:
        print('No Records')
        continue

In [ ]:
df.to_csv('./csv/NF Physician.csv', index=False)